In [1]:
# Notebook for prediction and evaluation of multi-step forecasting DMSLSTM models
# for the most recent training system, based on a single JSON configuration file

In [2]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf

# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

In [4]:
# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
# therefore, get and test the appropriate function
from tensorflow.saved_model import load

In [5]:
# symmetrical mean absolute percentage error
def smape(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

### inference process is driven by the saved model, the corresponding SLDB and the dataset

In [8]:
# this code block will be imported as:
# from dmslstm.data import _parse_dataset_function
read_features = {
    'hourly': tf.io.VarLenFeature(dtype=tf.float32),
    'daily': tf.io.VarLenFeature(dtype=tf.float32),
    'weekly': tf.io.VarLenFeature(dtype=tf.float32),
    'target': tf.io.VarLenFeature(dtype=tf.float32),
    'oh_wd': tf.io.VarLenFeature(dtype=tf.float32),
    'oh_dh': tf.io.VarLenFeature(dtype=tf.float32),
    'timestamp': tf.io.VarLenFeature(dtype=tf.string)
}


def _parse_dataset_function(example_proto, objective_shapes, parse_timestamp):
    # parse the input tf.Example proto using the dictionary above
    row = tf.io.parse_single_example(example_proto, read_features)
    # pass objective shape as a list of lists [hourly_shape, daily_shape, weekly_shape]
    hourly = tf.reshape(row['hourly'].values, objective_shapes['hourly'])
    daily = tf.reshape(row['daily'].values, objective_shapes['daily'])
    weekly = tf.reshape(row['weekly'].values, objective_shapes['weekly'])
    target = tf.reshape(row['target'].values, objective_shapes['target'])
    oh_wd = tf.reshape(row['oh_wd'].values, objective_shapes['oh_wd'])
    oh_dh = tf.reshape(row['oh_dh'].values, objective_shapes['oh_dh'])
    # do not parse the timestamp to TPUEstimator, as it does not support string types!
    # ToDo: code timestamps into features, as numbers
    #  so they can be parsed for training, if needed later
    timestamp = tf.reshape(row['timestamp'].values, objective_shapes['timestamp'])
    # the parsed dataset must have the shape {features}, target!!!
    # so:
    feature_dict = {
        'hourly': hourly,
        'daily': daily,
        'weekly': weekly,
        'oh_wd': oh_wd,
        'oh_dh': oh_dh,
    }
    # Do not parse the timestamp for training!!! Strings are not supported in TPUs!!!,
    # or parse it as a number
    if parse_timestamp:
        feature_dict['timestamp'] = timestamp

    return feature_dict, target

In [6]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [21]:
# during batch prediction, the model identifier is obtained via Abseil Flags
# remember this notebook is based on local execution,
# therefore model directory must be downloaded from GS before running the notebook
model_id = 'DMSLSTM_TPU_007'

# during batch prediction, the SLDB identifier is obtained via Abseil Flags
# THE SLDB FOR INFERENCE MUST BE THE SAME USED FOR TRAINING! (THE ONE SETUP IN THE CONFIGURATION FILE)
sldb_id = 'CPE04115_H_kw_20210526212214_008001_008024_008168_048'

# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

# ADD AN INFERENCE IDENTIFIER, BECAUSE FOR ARTRFDC MODELS, DIFFERENT INFERENCES
# CAN BE PRODUCED FROM A SINGLE SAVED MODEL (USUALLY DIFFERENT FORECAST WINDOWS)
# during batch prediction, the inference identifier should be obtained via Abseil Flags
inference = '048'

### run inference on model_id and dataset for given executions

In [22]:
# during batch prediction, the execution identifier is obtained via Abseil Flags
for execution in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    
    # use model identifier and execution number to build the model directory string
    model_dir = '{}_{:02d}'.format(model_id, execution)

    # get the path to the saved model main directory
    saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                         'models',
                                                         model_dir)

    # get all the files in the saved model path, to find the most recent one
    all_files = os.listdir(saved_model_path)
    # get the path to the most recent saved model
    latest_saved_model_id = sorted(all_files)[-1]
    # build the full path for the latest saved model dir
    export_dir = '{}/{}'.format(saved_model_path, latest_saved_model_id)
    print ('Exported model path is {}'.format(export_dir))

    # load the saved model and the prediction function
    imported = load(export_dir=export_dir, tags='serve')
    predict_fn = imported.signatures["serving_default"]

    # build a path to the sldb directory
    data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

    # then get the ts_identifier from the json file in the sldb directory
    sldb_json_file = '{}/sldb.json'.format(data_dir)

    # open the json file
    with open(sldb_json_file, 'r') as inputfile:
        sldb_dict = json.load(inputfile)

    # and get the time series identifier
    ts_identifier = sldb_dict['ts']

    # use the time series identifier to obtain the SK-Learn scaler used on it,
    # remember this is usually the scaler for the test dataset (unseen data)
    scaler = joblib.load('{}/{}/{}/scaler_{}.save'.format(PROJECT_ROOT,
                                                          'timeseries',
                                                          ts_identifier,
                                                          dataset))

    print('Scaler loaded for time series {} and {} dataset'.format(ts_identifier, dataset))

    # build a path to the dataset for prediction
    dataset_path = '{}/{}.tfrecord'.format(data_dir, dataset)
    # load the dataset
    tfrecord_dataset = tf.data.TFRecordDataset(dataset_path)

    # a list to store prediction values
    predictions_list = list()

    # TensorFlow 2 eager execution allows to iterate over a dataset
    for element in tfrecord_dataset:
        predictions_list.append(predict_fn(element))

    # get prediction values from predictions list
    predictions = [p['forecast'][0] for p in predictions_list]

    # pass predictions to a NumPy array
    predictions_array = np.asarray(predictions)

    # array from shape (rows, timesteps, 1) to (rows, timesteps)
    predictions_array = np.squeeze(predictions_array)

    # inverse-scale predictions
    rescaled_predictions = scaler.inverse_transform(predictions_array)

    # temporarily skip JSON serialization of predictions and targets for multistep forecasting

    # get the SLDB parameters for the forecasting model
    # parameters_json_file = '{}/{}/{}/sldb_parameters.json'.format(PROJECT_ROOT,
    #                                                               'parameters',
    #                                                               model_id)
    
    # get the SLDB parameters from the JSON configuration file
    config_json_file = '{}/{}/{}.json'.format(PROJECT_ROOT,
                                              'parameters',
                                              model_id)    

    # recover the sldb dictionary from the json file in parameters/
    # with open(parameters_json_file, 'r') as inputfile:
    #     sldb_parameters = json.load(inputfile)

    # recover the sldb dictionary from the json file in parameters/
    with open(config_json_file, 'r') as inputfile:
        configuration = json.load(inputfile)
    
    # store the objective shapes for reshaping tensors in a dictionary
    _EXTRACTING_OBJECTIVE_SHAPES = {
        'hourly': [configuration['embedding']['hourly'], 1],
        'daily': [configuration['embedding']['daily'], 1],
        'weekly': [configuration['embedding']['weekly'], 1],
        'target': [configuration['no_targets'], 1],
        'oh_wd': [7, 1],  # Monday to Sunday
        'oh_dh': [24, 1],  # midnight to 23:00
        'timestamp': [configuration['no_targets'], 1]
    }

    # test_dataset was previously acquired from tfrecord file
    # use it again to build arrays for targets and timestamps
    parsed_dataset = tfrecord_dataset.map(
        lambda row: _parse_dataset_function(
            example_proto=row,
            objective_shapes=_EXTRACTING_OBJECTIVE_SHAPES,
            parse_timestamp=True
        )
    )

    # a list to store the string_timestamps
    string_timestamps_list = list()

    # a list to store the targets
    targets_list = list()

    # get string_timestamps and targets associated to the predictions previously served
    for parsed_example in parsed_dataset:
        string_timestamps = np.squeeze(np.asarray(parsed_example[0]['timestamp']).astype(str))
        string_timestamps_list.append(string_timestamps)
        targets = np.squeeze(parsed_example[1])
        targets_list.append(targets)


    # get the number of rows in the dataset for prediction
    length = configuration['total_{}_rows'.format(dataset)]
    print('Number of rows in the {} dataset is {}.'.format(dataset, length))

    # confirm all string_timestamps were loaded
    print('Loaded all string timestamps: {}'.format(
        len(string_timestamps_list) == length)
    )

    # confirm all targets were loaded
    print('Loaded all targets: {}'.format(
        len(targets_list) == length)
    )

    # targets to array
    targets_array = np.asarray(targets_list)

    # rescale the targets
    rescaled_targets = scaler.inverse_transform(targets_array)

    # a columns list for the predictions dataframe
    pred_df_columns = ['model_id',
                       'execution',
                       'dataset',
                       'inference',
                       'string_timestamps',
                       'predictions',
                       'targets']

    # a list with model_id repeated length times, to populate the predictions detail dataframe
    model_id_repeat_list = [model_id]*length
    # same for execution
    execution_repeat_list = [execution]*length
    # same for dataset
    dataset_repeat_list = [dataset]*length
    # same for the inference identifier
    inference_repeat_list = [inference]*length

    # predictions dataframe
    predictions_detail_df = pd.DataFrame(list(zip(model_id_repeat_list,
                                                  execution_repeat_list,
                                                  dataset_repeat_list,
                                                  inference_repeat_list,
                                                  # from 2D NumPy array to list of 1D arrays
                                                  string_timestamps_list,
                                                  # from 2D NumPy array to list of 1D arrays
                                                  rescaled_predictions.tolist(),
                                                  rescaled_targets.tolist())), columns=pred_df_columns)

    # complement the detailed predictions dataframe with mae, rmse, smape
    # row by row, will be averaged at model-execution level, later...

    # a list with MAE, evaluated row by row
    predictions_detail_df['mae'] = [mean_absolute_error(row.targets, row.predictions) \
                                    for _, row in predictions_detail_df.iterrows()]

    # a list with RMSE, evaluated row by row
    predictions_detail_df['rmse'] = [sqrt(mean_squared_error(row.targets, row.predictions)) \
                                     for _, row in predictions_detail_df.iterrows()]

    # a list with SMAPE, evaluated row by row
    predictions_detail_df['smape'] = [smape(row.targets, row.predictions) \
                                      for _, row in predictions_detail_df.iterrows()]

    # build a predictions summary dataframe, reset index to avoid making a multi-column index when grouping by
    predictions_summary_df = predictions_detail_df.groupby(['model_id',
                                                            'execution',
                                                            'dataset',
                                                            'inference']).mean().reset_index()

    # a range to iterate on prediction timesteps
    targets_range = np.arange(configuration['no_targets'])

    # vector metric (vector component to vector component)
    # an array no_targets-d: metric for 1, 2,..., no_targets step-ahead (target versus prediction for rows in dataset)

    # for index, row in dataframe.iterrows()
    mae_vector = [
        mean_absolute_error(
            # a list with the n-rows target values for the n-th step ahead
            [row.targets[n] for _, row in predictions_detail_df.iterrows()],
            # a list with the n-rows prediction values for the n-th step ahead
            [row.predictions[n] for _, row in predictions_detail_df.iterrows()]
        ) for n in targets_range
    ]
    predictions_summary_df['mae_vector'] = [mae_vector]

    # for index, row in dataframe.iterrows()
    rmse_vector = [
        sqrt(mean_squared_error(
            # a list with the n-rows target values for the n-th step ahead
            [row.targets[n] for _, row in predictions_detail_df.iterrows()],
            # a list with the n-rows prediction values for the n-th step ahead
            [row.predictions[n] for _, row in predictions_detail_df.iterrows()]
        )) for n in targets_range
    ]
    predictions_summary_df['rmse_vector'] = [rmse_vector]

    # for index, row in dataframe.iterrows()
    smape_vector = [
        smape(
            [row.targets[n] for _, row in predictions_detail_df.iterrows()],
            [row.predictions[n] for _, row in predictions_detail_df.iterrows()]
        ) for n in targets_range
    ]
    predictions_summary_df['smape_vector'] = [smape_vector]

    # insert count of rows as a column value
    predictions_summary_df.insert(4, 'count', length)

    # build a path to persist the dataframe to database/predictions_detail/
    detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}.pkl'.format(
        PROJECT_ROOT,
        'database',
        'predictions_detail',
        model_id,
        execution,
        dataset,
        inference)

    # persist the Pandas dataframe to database/predictions_detail/
    predictions_detail_df.to_pickle(detail_pickle_path)
    print('Persisted Pandas dataframe for predictions detail of {}_{:02d}_{}_{}'.format(model_id,
                                                                                        execution,
                                                                                        dataset,
                                                                                        inference))

    # build a path to persist the dataframe to database/predictions_summary/
    summary_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}.pkl'.format(
        PROJECT_ROOT,
        'database',
        'predictions_summary',
        model_id,
        execution,
        dataset,
        inference)

    # persist the Pandas dataframe to database/predictions_summary/
    predictions_summary_df.to_pickle(summary_pickle_path)
    print('Persisted Pandas dataframe for predictions summary of {}_{:02d}_{}_{}'.format(model_id,
                                                                                         execution,
                                                                                         dataset,
                                                                                         inference))


Exported model path is /home/developer/gcp/cbidmltsf/models/DMSLSTM_TPU_007_00/export/exporter/1623433248
Scaler loaded for time series CPE04115_H_kw_20210526212214 and test dataset
Number of rows in the test dataset is 817.
Loaded all string timestamps: True
Loaded all targets: True
Persisted Pandas dataframe for predictions detail of DMSLSTM_TPU_007_00_test_048
Persisted Pandas dataframe for predictions summary of DMSLSTM_TPU_007_00_test_048
Exported model path is /home/developer/gcp/cbidmltsf/models/DMSLSTM_TPU_007_01/export/exporter/1623433421
Scaler loaded for time series CPE04115_H_kw_20210526212214 and test dataset
Number of rows in the test dataset is 817.
Loaded all string timestamps: True
Loaded all targets: True
Persisted Pandas dataframe for predictions detail of DMSLSTM_TPU_007_01_test_048
Persisted Pandas dataframe for predictions summary of DMSLSTM_TPU_007_01_test_048
Exported model path is /home/developer/gcp/cbidmltsf/models/DMSLSTM_TPU_007_02/export/exporter/162343360